**Note to grader:** Each question is assigned with a score. The final score will be (sum of actual scores)/(sum of maximum scores)*100. The grading rubrics are shown in the TA guidelines.

# **Assignment 4**

The goal of this assignment is to run some experiments with scikit-learn on a fairly sizeable and interesting image data set. This is the MNIST data set that consists of lots of images, each having 28x28 pixels. By today's standards, this may seem relatively tiny, but only a few years ago was quite challenging computationally, and it motivated the development of several ML algorithms and models that are now state-of-the-art  solutions for much bigger data sets.

The assignment is experimental. We will try to whether a combination of PCA and kNN can yield any good results for the MNIST data set. Let's see if it can be made to work on this data set.

Note: There are less difficult Python parts in this assignment. You can get things done by just repeating things from the class notebooks. But your participation and interaction via Canvas is always appreciated!

## Preparation Steps

In [ ]:
# Import all necessary python packages
import numpy as np
#import os
#import pandas as pd
#import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap
#from sklearn.linear_model import LogisticRegression

In [ ]:
# we load the data set directly from scikit learn
#
# note: this operation may take a few seconds. If for any reason it fails we
# can revert back to loading from local storage.

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
y = y.astype(int)
X = ((X / 255.) - .5) * 2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=10000, random_state=123, stratify=y)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## <font color = 'blue'> Question 1. Inspecting the Dataset （50 pts = 10 pts by 5 questions)</font>

**(i)** How many data points are in the training and test sets ? <br>
**(ii)** How many attributes does the data set have ?

Exlain how you found the answer to the first two questions.

[**Hint**: Use the 'shape' method associated with numpy arrays. ]

**(iii)** How many different labels does this data set have. Can you demonsrate how to read that number from the vector of labels *y_train*?  <br>
**(iv)** How does the number of attributes relates to the size of the images? <br>
**(v)** What is the role of line 12 (X = ((X / 255.) - .5) * 2) in the above code?





*(Please insert cells below for your answers. Clearly id the part of the question you answer)*

In [ ]:
print("This for Question 1 part i")
print(f"The length of data points for training sets are {len(X_train)}")
print(f"The length of data points for test sets are {len(X_test)}")

This for Question 1 part i
The length of data points for training sets are 60000
The length of data points for test sets are 10000


In [ ]:
print("This for Question 1 part ii\n")
print("For the attributes of that dataset we can check the number of columns in the dataset since\nthey are attributes and using .shape we can get the number of dimensions in the numpy array\n")
print(f"The number of attributes in the dataset is {X_train.shape[1]}")

This for Question 1 part ii

For the attributes of that dataset we can check the number of columns in the dataset since
they are attributes and using .shape we can get the number of dimensions in the numpy array

The number of attributes in the dataset is 784


In [ ]:
print("This for Question 1 part iii\n")
print(f"To see how many labels we have would be to see how many different outcomes we can get, and to not get any repeats we use unique to get the unique values and the labels are {y_train.unique()} and there are {len(y_train.unique())} labels")

This for Question 1 part iii

To see how many labels we have would be to see how many different outcomes we can get, and to not get any repeats we use unique to get the unique values and the labels are [4 8 7 0 5 2 6 9 3 1] and there are 10 labels


In [ ]:
print("This for Question 1 part iv\n")
print("The number of attributes is related to the size of the image since the image is 28 by 28 pixels and if we multiply the height and width of the image we get the number of attributes which is 784")

This for Question 1 part iv

The number of attributes is related to the size of the image since the image is 28 by 28 pixels and if we multiply the height and width of the image we get the number of attributes which is 784


In [ ]:
print("This for Question 1 part v\n")
print("The role of line 12 (X = ((X / 255.) - .5) * 2) is to decrease the size of the array so the values are not so large and to make the training space a bit smaller")

This for Question 1 part v

The role of line 12 (X = ((X / 255.) - .5) * 2) is to decrease the size of the array so the values are not so large and to make the training space a bit smaller


In [ ]:
# For grader use only
maxScore = maxScore + 50


##  <font color = 'blue'> Question 2. PCA on MNIST (10 pts) </font>

Because the number of attributes of the MNIST data set may be too big to apply kNN on it (due to the 'curse of dimensionality'), we want to compress the images down to a smaller number of 'fake' attributes.

Use scikit-learn to output a data set *X_train_transformed* and *X_test_transformed*, with $l$ attributes. Here a reasonable choice of $l$ is 10, equal to the number of labels. But you can try slightly smaller or bigger values as well.

Print out the shape of *X_train_transformed* and *X_test_transformed*.


**Hint**: Take a look at [this notebook](https://colab.research.google.com/drive/1DG5PjWejo8F7AhozHxj8329SuMtXZ874?usp=drive_fs), and imitate what we did there. Be careful though, to use only the scikit-learn demonstration, not the exhaustive PCA steps.

**Note**: This computation can take a while. If problems are encountered we can try the same experiment on a downsized data set.

In [ ]:
from sklearn.decomposition import PCA
l = 10 #number of attributes recommended since that is the number of labels we have
pca = PCA(n_components=l) #using PCA will allow for a dimensionality reduction
X_train_transformed = pca.fit_transform(X_train) #transform train and test X with the l attributes
X_test_transformed = pca.transform(X_test)
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(60000, 10)
(10000, 10)


In [ ]:
# for grader use
maxScore = maxScore + 10



## <font color = 'blue'> Question 3. kNN on MNIST attributes from PCA （40 pts = 10 pts by 4 questions) </font>


Having calculated the *transformed* MNIST data set we can now apply a kNN approach to the MNIST classification data set. Here are the sets:

(i) Fit a $k$-NN classifier on the transformed data set. Here $k$ is a hyperparameter, and you can experiment with it. Be aware though, that larger $k$ can take more time to fit.

(ii) Apply the classifier on the transformed test set. What is the classification accuracy?

(iii) A theoretical question: if we skipped all the above steps and we just assigned a **random** label to each test point, what would the classification accuracy be on average?  Does your result (ii) beat the random expectation? (conduct 1000 trials to get the average accuracy)

(iv) Experiment with different settings of $k$. Experiment design: calculates accuracy for increasing values of k; stops when k decreases for 5 values of k; report your findings in a separate cell.

[**Hint**: Take a look at this [notebook](https://colab.research.google.com/drive/1Mh6I3bR8pE90kcs28JfKok59NtfV_7ct?usp=drive_fs)]


In [ ]:
print("Question 3 Part i and ii\n")
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4,p=2)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"The accurary of the classification on the transformed data set is {accuracy}")
print("")
print("Question 3 Part iii(1)\n")
print("If you assigned a random label to each test point then you would get a classification accuracy of 10% since if you're randomly guessing then you're likely to get one of the ten options correct")
print("Question 3 Part iii(2)\n")
avg_acc=0
for i in range(1000):
  knn = KNeighborsClassifier(n_neighbors=4,p=2)
  knn.fit(X_train_transformed, y_train)
  y_pred = knn.predict(X_test_transformed)
  accuracy = accuracy_score(y_test, y_pred)
  avg_acc+=accuracy
print(f"The average accuracy for 1000 trials to see if Knn would beat our random expectation is {avg_acc/1000}")

Question 3 Part i and ii

The accurary of the classification on the transformed data set is 0.9314

Question 3 Part iii(1)

If you assigned a random label to each test point then you would get a classification accuracy of 10% since if you're randomly guessing then you're likely to get one of the ten options correct
Question 3 Part iii(2)

The average accuracy for 1000 trials to see if Knn would beat our random expectation is 0.9314000000000222


In [ ]:
print("Question 3 Part iv\n")
knn = KNeighborsClassifier(n_neighbors=7,p=2)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"The accurary of the classification on the transformed data set for k=7 is {accuracy}")
knn = KNeighborsClassifier(n_neighbors=2,p=2)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"The accurary of the classification on the transformed data set for k=2 is {accuracy}")
print("The accuracy reported to be slighly higher with k increased over time")

Question 3 Part iv

The accurary of the classification on the transformed data set for k=7 is 0.9344
The accurary of the classification on the transformed data set for k=2 is 0.9147
The accuracy reported to be slighly higher with k increased over time


In [ ]:
# for grader use
maxScore = maxScore + 40



In [ ]:
# for grader use

score = actualScore*100/maxScore